In [1]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf PyMPDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/PyMPDATA.git
    %cd PyMPDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from PyMPDATA_examples.Olesik_et_al_2020.setup import Setup, default_opt_set, option_string
from PyMPDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from PyMPDATA.options import Options
from PyMPDATA_examples.Olesik_et_al_2020.simulation import Simulation
from PyMPDATA_examples.utils.show_plot import show_plot
from PyMPDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from PyMPDATA.arakawa_c.discretisation import discretised_analytical_solution
from PyMPDATA_examples.utils.error_norms import L2, Smolarkiewicz_Grabowski_1990_eq21, modified_Smolarkiewicz_Rasch_r0
from PyMPDATA_examples.Olesik_et_al_2020.analysis import rel_disp
from joblib import Parallel, parallel_backend, delayed

In [6]:
CI = 'TRAVIS' in os.environ
GCs = np.linspace(.05,.95, 3) if CI else np.linspace(.05,.95, 7)
nrs = np.array([2**n for n in (range(7, 10) if CI else range(7,15))], dtype=int)

In [7]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [8]:
const = min(np.log2(1/nrs)) - 1


def phi(cour):
    return cour * np.pi / 2

def rho(n):
    return np.log2(1/n) - const


def polar_plot(nr, cour, values, name):
    theta_array = phi(cour)
    r_array = rho(nr)

    X, Y = np.meshgrid(theta_array, r_array)
    Z = np.array(list(values)).reshape(len(r_array), len(theta_array))

    min_val = -26
    max_val = -2

    amplitude = max_val - min_val
    if name == 'log$_2$(err)':
        levels = np.linspace(
            min_val,
            max_val,
            int(amplitude + 1)
        )
    else:
        levels = 7

    ax = plt.subplot(111, projection='polar')
    cnt = ax.contourf(X, Y, Z, levels, cmap='jet')
    plt.contour(X, Y, Z, levels, colors='black')
    ax.scatter(X, Y, alpha=.8, s=10)
    legend = plt.colorbar(cnt, ax=ax, pad=0.1)
    legend.set_label(r'$log_2(Err_{L2})$', rotation=90)
    ax.set_thetalim(min(theta_array),max(theta_array))
    ax.set_rlim(0, max(r_array))
    
    ticks = r_array
    ax.set_yticks(ticks)
    ax.set_yticklabels("$2^{" + f"{int(-tick - const):d}" + "}$" for tick in ticks)
    
    ax.set_thetagrids(cour * 90, tuple(f"{c:.2g}" for c in cour))
    ax.grid(True)
    ax.set_xlabel(r"$nx$", labelpad=18)
    ax.annotate(r'$C$', xy=(.8, .85), xycoords='axes fraction')

In [9]:
def analysis(setup, GC, opt):
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    def pdf_arg(r):
        return pdf_t(r* rh.units).magnitude
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                pdf_arg, midpoint_value = True,
                r=simulation.r.magnitude
            ) * pdf_t(rh[0]).units
    numerical = simulation.n_of_r
    psi = simulation.psi
    g_factor = simulation.g_factor
    dp_dr = simulation.dp_dr
    
    maximum_numeric = np.max(numerical)
    maximum_analytic = np.max(analytical)
    measure_height = (maximum_numeric / maximum_analytic).magnitude
    
    dif = analytical-numerical
    measure_h_2 =  (1/t * np.sqrt( 1/len(dif) * dif.dot(dif))).magnitude
  
    error = np.log2(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude))
    error_g = np.log2(Smolarkiewicz_Grabowski_1990_eq21(g_factor * psi.magnitude, g_factor * analytical.magnitude / dp_dr, t.magnitude))
    error_r0 = np.log2(modified_Smolarkiewicz_Rasch_r0(psi.magnitude, analytical.magnitude /dp_dr, t.magnitude, g_factor))
    return setup.nr, GC, error

In [10]:
for opt in default_opt_set.values():
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(Setup(nr = nr, mixing_ratios_g_kg = [1.05,]), GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = tuple(tuple(i) for i in zip(*results0))
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2]}
    opt=option_string(str(opt))
    print(opt)
    for measure in measures.keys():
        polar_plot(nrs, GCs, measures[measure], name = measure)
        show_plot(filename = f'convergence_{measure}_{opt}.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
/usr/lib/python3/dist-packages/pint/quantity.py:1377: UnitStrippedWarning: The unit of the quantity is stripped.
  warnings.warn("The unit of the quantity is stripped.", UnitStrippedWarning)
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:   40.1s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:   55.8s
[Parallel(n_jobs=-2)]: Done  49 out of  56 | elapsed:  1.4min remaining:   12.3s
[Parallel(n_jobs=-2)]: Done  56 out of  56 | elapsed:  2.3min finished
<ipython-input-8-7ec00f8c6b2b>:33: UserWarning: The following kwargs were not used by contour: 'linewidth'
  plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)


upwind


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
/usr/lib/python3/dist-packages/pint/quantity.py:1377: UnitStrippedWarning: The unit of the quantity is stripped.
  warnings.warn("The unit of the quantity is stripped.", UnitStrippedWarning)
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   33.2s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-2)]: Done  49 out of  56 | elapsed:  1.6min remaining:   13.3s


MPDATA 2 iterations


[Parallel(n_jobs=-2)]: Done  56 out of  56 | elapsed:  2.4min finished
<ipython-input-8-7ec00f8c6b2b>:33: UserWarning: The following kwargs were not used by contour: 'linewidth'
  plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)


...

MPDATA 3 iterations


<ipython-input-8-7ec00f8c6b2b>:33: UserWarning: The following kwargs were not used by contour: 'linewidth'
  plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
/usr/lib/python3/dist-packages/pint/quantity.py:1377: UnitStrippedWarning: The unit of the quantity is stripped.
  warnings.warn("The unit of the quantity is stripped.", UnitStrippedWarning)
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-2)]: Done  49 out of  56 | elapsed:  1.6min remaining:   13.5s
[Parallel(n_jobs=-2)]: Done  56 out of  56 | elapsed:  2.5min finished


MPDATA 3 iterations third order terms infinite gauge non-oscillatory


<ipython-input-8-7ec00f8c6b2b>:33: UserWarning: The following kwargs were not used by contour: 'linewidth'
  plt.contour(X, Y, Z, levels, colors='black', linewidth=.3)
